<a href="https://colab.research.google.com/github/ccwu0918/book-sqlfifty/blob/main/ch07-where/ch07-where.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL 的五十道練習：初學者友善的資料庫入門

> 篩選觀測值

讀者如果是資料科學的初學者，可以略過下述的程式碼；讀者如果不是資料科學的初學者，欲使用 JupyterLab 執行本章節內容，必須先執行下述程式碼載入所需模組與連接資料庫。

In [ ]:
!git clone https://github.com/ccwu0918/book-sqlfifty

In [ ]:
# %LOAD sqlite3 db=../databases/imdb.db timeout=2 shared_cache=true

In [ ]:
import sqlite3
import unittest
import json
import os
import numpy as np
import pandas as pd
conn = sqlite3.connect('./databases/imdb.db')
conn.execute("""ATTACH './databases/covid19.db' AS covid19""")
conn.execute("""ATTACH './databases/twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH './databases/nba.db' AS nba""")
conn.execute("""ATTACH './databases/northwind.db' AS Northwind""")
conn.execute("""ATTACH './databases/Chinook_Sqlite.sqlite' AS Chinook""")

In [ ]:
# %%capture
# load the SQL magic extension
# https://github.com/catherinedevlin/ipython-sql
# this extension allows us to connect to DBs and issue SQL command
%load_ext sql

# now we can use the magic extension to connect to our SQLite DB
# use %sql to write an inline SQL command
# use %%sql to write SQL commands in a cell
%sql sqlite:///databases/imdb.db

In [ ]:
%%sql
ATTACH "./databases/covid19.db" AS covid19;
ATTACH "./databases/twElection2020.db" AS twElection2020;
ATTACH "./databases/nba.db" AS nba;
ATTACH "./databases/northwind.db" AS Northwind;
ATTACH "./databases/Chinook_Sqlite.sqlite" AS Chinook;

In [ ]:
%%sql
SELECT sqlite_version();

## 複習一下

在第四章「衍生計算欄位」我們提過關係運算符與邏輯運算符在後續的「篩選觀測值」以及「條件邏輯」的章節中佔有舉足輕重的地位，針對常數或欄位可以使用關係運算符衍生計算欄位，應用後會得到 0（布林值 `False`）或 1（布林值 `True`）兩者其中之一，基礎的關係運算符有：

|關係運算符|作用描述|
|:-------|:------|
|`=`|相等|
|`!=`|不相等|
|`>`|大於|
|`>=`|大於等於|
|`<`|小於|
|`<=`|小於等於|
|`LIKE`|相似|
|`IN`|存在於|
|`BETWEEN lower_bound AND upper_bound`|大於等於 `lower_bound` 且小於等於 `upper_bound`|
|`IS NULL`|是否為空值|

運用關係運算符於資料表的欄位，這時所形成的布林值就會隨著列數而產生基於列（Row-wise）的運算。比較結果為布林值 `False`，SQLite 以 0 表示；比較結果為布林值 `True`，SQLite 以 1 表示。

In [ ]:
%%sql
SELECT release_year,
       release_year = 1994 AS released_in_1994
  FROM movies
 LIMIT 10;

release_year,released_in_1994
1994,1
1972,0
2008,0
1974,0
1957,0
1993,0
2003,0
1994,1
2001,0
1966,0


## 以 `WHERE` 從資料表篩選

在第三章「從資料表選擇」運用 `SELECT` 保留字搭配欄位名稱可以取出資料表中指定的欄，這個章節我們要運用 `WHERE` 保留字搭配條件（Conditions）取出資料表符合「條件」的觀測值。

```sql
SELECT DISTINCT columns AS alias
  FROM table
 WHERE conditions
 ORDER BY columns DESC
 LIMIT m;
```

當我們使用關係運算符衍生計算欄位，應用後所得到的 0（布林值 `False`）或 1（布林值 `True`）就是所謂的「條件」，將衍生計算所得的布林值放在 `WHERE` 保留字之後會將 1（布林值 `True`）的觀測值留在查詢結果中，就能完成從資料表篩選的任務。

In [ ]:
%%sql
SELECT release_year,
       release_year = 1994 AS released_in_1994
  FROM movies
 WHERE release_year = 1994;

release_year,released_in_1994
1994,1
1994,1
1994,1
1994,1
1994,1


在一段 SQL 敘述中同時選擇指定欄位、符合條件觀測值，稱為子集（Subset），指的是從原本外型為 `(m, n)` 的資料表中選取出了一個列數較少、欄數較少的查詢結果。

In [ ]:
%%sql
SELECT title,
       release_year,
       rating,
       runtime
  FROM movies
 WHERE release_year = 1994;

title,release_year,rating,runtime
The Shawshank Redemption,1994,9.3,142
Pulp Fiction,1994,8.9,154
Forrest Gump,1994,8.8,142
Léon: The Professional,1994,8.5,110
The Lion King,1994,8.5,88


關係運算符也可以應用在文字資料類別的欄位，像是 `=` 作為文字內容的精準比對。

In [ ]:
%%sql
SELECT title,
       release_year,
       director
  FROM movies
 WHERE director = 'Christopher Nolan'
 ORDER BY release_year;

title,release_year,director
Memento,2000,Christopher Nolan
Batman Begins,2005,Christopher Nolan
The Prestige,2006,Christopher Nolan
The Dark Knight,2008,Christopher Nolan
Inception,2010,Christopher Nolan
The Dark Knight Rises,2012,Christopher Nolan
Interstellar,2014,Christopher Nolan


## 文字特徵比對

除了使用 `=` 作為作為文字內容精準比對的關係運算符，我們也常需要使用具備特徵比對（Pattern matching）性質的關係運算符 `LIKE`，使用 `LIKE` 作為關係運算符的時候要搭配萬用字元（Wildcards）來描述特徵。

|萬用字元|作用描述|
|:------|:-----|
|`%`|表示任意文字，包含空字元|
|`_`|表示剛好一個文字|

舉例來說，`'The Lord of the Rings%'` 這個文字特徵表示開頭為 Lord of the Rings 後面接任意文字。

In [ ]:
%%sql
SELECT title
  FROM movies
 WHERE title LIKE 'The Lord of the Rings%'; -- The Lord of the Rings followed by any characters

title
The Lord of the Rings: The Return of the King
The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings: The Two Towers


舉例來說，`'The ______'` 這個文字特徵表示開頭為 The 後面接一個空白、六個任意文字。

In [ ]:
%%sql
SELECT title
  FROM movies
 WHERE title LIKE 'The ______'; -- The followed by a space and 6 characters

title
The Matrix
The Father


## `WHERE` 後的多個條件

當 `WHERE` 保留字搭配多個條件時，我們會需要「邏輯運算符」結合這些布林值，基礎的邏輯運算符有：

|邏輯運算符|作用描述|
|:-------|:------|
|`AND`|和，交集|
|`OR`|或，聯集|
|`NOT`|反轉布林值，將 0（布林值 `False`）與 1（布林值 `True`）互換|

使用 `AND` 結合兩個條件時，要兩條件皆為真才會判斷為真，其餘狀況均為假。

In [ ]:
%%sql
SELECT 0 AND 0 AS False,
       0 AND 1 AS False,
       1 AND 0 AS False,
       1 AND 1 AS True;

False,False,False,True
0,0,0,1


將 `AND` 應用在資料表欄位，所形成的判斷結果也會隨著列數而產生基於列的運算，因此最後從資料表中篩選的觀測值，會是 `condition_1_and_condition_2` 為 1（布林值 `True`）的觀測值。

In [ ]:
%%sql
SELECT release_year >= 1990 AS condition_1,
       release_year <= 2010 AS condition_2,
       release_year >= 1990 AND release_year <= 2010 AS condtion_1_and_condition_2
  FROM movies
 LIMIT 5;

condition_1,condition_2,condtion_1_and_condition_2
1,1,1
0,1,0
1,1,1
0,1,0
0,1,0


當我們以 `AND` 結合 `>=` 與 `<=` 兩個關係運算符之條件時，可以改使用 `BETWEEN lower_bound AND upper_bound` 作為關係運算符。

In [ ]:
%%sql
SELECT release_year >= 1990 AS condition_1,
       release_year <= 2010 AS condition_2,
       release_year BETWEEN 1990 AND 2010 AS condtion_1_and_condition_2
  FROM movies
 LIMIT 5;

condition_1,condition_2,condtion_1_and_condition_2
1,1,1
0,1,0
1,1,1
0,1,0
0,1,0


In [ ]:
%%sql
SELECT title,
       release_year
  FROM movies
 WHERE release_year BETWEEN 1990 AND 2010
 LIMIT 5;

title,release_year
The Shawshank Redemption,1994
The Dark Knight,2008
Schindler's List,1993
The Lord of the Rings: The Return of the King,2003
Pulp Fiction,1994


使用 `OR` 結合兩個條件時，要兩者皆為假才為假，其餘狀況均為真。

In [ ]:
%%sql
SELECT 0 OR 0 AS False,
       0 OR 1 AS True,
       1 OR 0 AS True,
       1 OR 1 AS True;

False,True,True,True
0,1,1,1


將 `OR` 應用在資料表欄位，所形成的判斷結果也會隨著列數而產生基於列的運算，因此最後從資料表中篩選的觀測值，會是 `condition_1_or_condition_2` 為 1（布林值 `True`）的觀測值。

In [ ]:
%%sql
SELECT director = 'Steven Spielberg' AS condition_1,
       director = 'Christopher Nolan' AS condition_2,
       director = 'Steven Spielberg' OR director = 'Christopher Nolan' AS condtion_1_or_condition_2
  FROM movies
 LIMIT 10;

condition_1,condition_2,condtion_1_or_condition_2
0,0,0
0,0,0
0,1,1
0,0,0
0,0,0
1,0,1
0,0,0
0,0,0
0,0,0
0,0,0


當我們以 `OR` 結合條件時，可以改使用 `IN` 作為關係運算符，並以小括號 `()` 表示欲判斷是否存在於的目標集合。

In [ ]:
%%sql
SELECT director = 'Steven Spielberg' AS condition_1,
       director = 'Christopher Nolan' AS condition_2,
       director IN ('Steven Spielberg', 'Christopher Nolan') AS condtion_1_or_condition_2
  FROM movies
 LIMIT 10;

condition_1,condition_2,condtion_1_or_condition_2
0,0,0
0,0,0
0,1,1
0,0,0
0,0,0
1,0,1
0,0,0
0,0,0
0,0,0
0,0,0


In [ ]:
%%sql
SELECT title,
       director
  FROM movies
 WHERE director IN ('Steven Spielberg', 'Christopher Nolan')
 LIMIT 5;

title,director
The Dark Knight,Christopher Nolan
Schindler's List,Steven Spielberg
Inception,Christopher Nolan
Saving Private Ryan,Steven Spielberg
Interstellar,Christopher Nolan


使用 `NOT` 可以將條件的判斷結果反轉，亦即真假互換。

In [ ]:
%%sql
SELECT NOT 1 AS False,
       NOT 0 AS True;

False,True
0,1


`NOT` 可以擺放在條件的前面，也可以放置在關係運算符之前，通常我們偏好唸起來與英文更相似的敘述，例如 `director NOT IN ('Steven Spielberg', 'Christopher Nolan')` 比 `NOT director IN ('Steven Spielberg', 'Christopher Nolan')` 更像英文，雖然查詢結果是相同的，但會使用前者這樣的敘述。

In [ ]:
%%sql
SELECT title,
       director
  FROM movies
 WHERE director NOT IN ('Steven Spielberg', 'Christopher Nolan') -- preferred
 LIMIT 5;

title,director
The Shawshank Redemption,Frank Darabont
The Godfather,Francis Ford Coppola
The Godfather Part II,Francis Ford Coppola
12 Angry Men,Sidney Lumet
The Lord of the Rings: The Return of the King,Peter Jackson


In [ ]:
%%sql
SELECT title,
       director
  FROM movies
 WHERE NOT director IN ('Steven Spielberg', 'Christopher Nolan')
 LIMIT 5;

title,director
The Shawshank Redemption,Frank Darabont
The Godfather,Francis Ford Coppola
The Godfather Part II,Francis Ford Coppola
12 Angry Men,Sidney Lumet
The Lord of the Rings: The Return of the King,Peter Jackson


## 空值的關係運算符

`NULL` 空值不適用 `=` 或者 `!=` 關係運算符，必須以 `IS` 作為關係運算符才能夠判斷是否為空值，若逕自使用 `=` 或者 `!=` 則判斷結果依然為 `NULL` 空值，無法得到 0（布林值 `False`）或 1（布林值 `True`）的判斷結果。

In [ ]:
%%sql
SELECT NULL = NULL AS null_value,
       NULL != NULL AS null_value,
       NULL IS NULL AS True,
       NULL IS NOT NULL AS False;

null_value,null_value,True,False
NULL,NULL,1,0


舉例來說，在 `covid19` 資料庫的 `lookup_table` 資料表中 `Province_State` 與 `Admin2` 欄都有不少空值的存在，如果對這兩欄分別使用 `=` 或者 `!=`，是無法得到 0（布林值 `False`）或 1（布林值 `True`）的判斷結果。

In [ ]:
%%sql
SELECT Province_State,
       Admin2,
       Province_State = NULL AS Province_State_equals_to_null,
       Admin2 != NULL AS Admin2_not_equals_to_null
  FROM lookup_table
 LIMIT 5;

Province_State,Admin2,Province_State_equals_to_null,Admin2_not_equals_to_null
NULL,NULL,NULL,NULL
NULL,NULL,NULL,NULL
NULL,NULL,NULL,NULL
NULL,NULL,NULL,NULL
American Samoa,NULL,NULL,NULL


必須以 `IS` 作為關係運算符才能夠判斷是否為空值。

In [ ]:
%%sql
SELECT Province_State,
       Admin2,
       Province_State IS NULL AS Province_State_is_null,
       Admin2 IS NOT NULL AS Admin2_is_not_null
  FROM lookup_table
 LIMIT 5;

Province_State,Admin2,Province_State_is_null,Admin2_is_not_null
NULL,NULL,1,0
NULL,NULL,1,0
NULL,NULL,1,0
NULL,NULL,1,0
American Samoa,NULL,0,0


In [ ]:
%%sql
SELECT Province_State,
       Admin2
  FROM lookup_table
 WHERE Province_State IS NOT NULL AND
       Admin2 IS NOT NULL
 LIMIT 5;

Province_State,Admin2
Sint Eustatius and Saba,Bonaire
Ascension and Tristan da Cunha,Saint Helena
Puerto Rico,Adjuntas
Puerto Rico,Aguada
Puerto Rico,Aguadilla


## 重點統整

- 使用關係運算符衍生計算欄位，應用後所得到的 0（布林值 `False`）或 1（布林值 `True`）就是所謂的「條件」，將衍生計算所得的布林值放在 `WHERE` 保留字之後會將 1（布林值 `True`）的觀測值留在查詢結果中，就能完成從資料表篩選的任務。
- 使用具備特徵比對（Pattern matching）性質的關係運算符 `LIKE`，使用 `LIKE` 作為關係運算符的時候要搭配萬用字元（Wildcards）來描述特徵。
- 這個章節學起來的 SQL 保留字：
    - `WHERE`
- 將截至目前所學的 SQL 保留字集中在一個敘述中，寫作順序必須遵從標準 SQL 的規定。

```sql
SELECT DISTINCT columns AS alias
  FROM table
 WHERE conditions
 ORDER BY columns DESC
 LIMIT m;
```

## 練習題 17-21

練習題會涵蓋四個學習資料庫，記得要依據題目的需求，調整編輯器選單的學習資料庫，在自己電腦的 SQLiteStudio 寫出跟預期輸出相同的 SQL 敘述，寫作過程如果卡關了，可以參考附錄二「練習題參考解答」。

### 17. 從 `covid19` 資料庫的 `time_series` 資料表將台灣的觀測值篩選出來，參考下列的預期查詢結果。

預期輸出：(861, 4) 的查詢結果。

In [ ]:
-- 礙於紙本篇幅僅顯示出前五列示意
%%sql


Country_Region,Date,Confirmed,Daily_Cases
Taiwan,2020-01-22,1,1
Taiwan,2020-01-23,1,0
Taiwan,2020-01-24,3,2
Taiwan,2020-01-25,3,0
Taiwan,2020-01-26,4,1


### 18. 從 `imdb` 資料庫的 `movies` 資料表將上映年份為 1994 的電影篩選出來，參考下列的預期查詢結果。

預期輸出：(5, 4) 的查詢結果。

In [ ]:
%%sql


title,rating,director,runtime
The Shawshank Redemption,9.3,Frank Darabont,142
Pulp Fiction,8.9,Quentin Tarantino,154
Forrest Gump,8.8,Robert Zemeckis,142
Léon: The Professional,8.5,Luc Besson,110
The Lion King,8.5,Roger Allers,88


### 19. 從 `imdb` 資料庫的 `actors` 資料表將 Tom Hanks、Christian Bale、Leonardo DiCaprio 篩選出來，參考下列的預期查詢結果。

註：Tom Hanks 是一位著名的美國男演員及電視製片人，以演技精湛而著稱。他是歷史上第2位連續兩屆獲得奧斯卡金像獎最佳男主角獎的演員，亦是最年輕獲得美國電影學會終身成就獎的演員。Christian Bale 是一名英國男演員和電影製片人，在蝙蝠俠三部曲中飾演 Bruce Wayne 獲得了廣泛讚揚及商業認可。Leonardo DiCaprio 是一位美國著名男演員、電影製片人兼環保概念的推動者，出演了由史詩愛情片鐵達尼號知名度大開。

來源: Wikipedia

預期輸出：(3, 2) 的查詢結果。

In [ ]:
%%sql


id,name
518,Christian Bale
1860,Leonardo DiCaprio
2957,Tom Hanks


### 20. 從 `imdb` 資料庫的 `movies` 資料表篩選出由 Christopher Nolan 或 Peter Jackson 所導演的電影，參考下列的預期查詢結果。

註：Christopher Nolan 是一名英國導演、編劇及監製，他的十部電影在全球獲得超過 47 億美元的票房，執導著名電影包含「黑暗騎士三部曲」、全面啟動、星際效應及敦克爾克大行動；Peter Jackson 是一名紐西蘭導演、編劇及監製，執導最出名的作品是「魔戒電影三部曲」與「哈比人電影系列」。

來源: Wikipedia

預期輸出：(10, 2) 的查詢結果。

In [ ]:
%%sql


title,director
The Dark Knight,Christopher Nolan
Inception,Christopher Nolan
Interstellar,Christopher Nolan
The Prestige,Christopher Nolan
Memento,Christopher Nolan
The Dark Knight Rises,Christopher Nolan
Batman Begins,Christopher Nolan
The Lord of the Rings: The Return of the King,Peter Jackson
The Lord of the Rings: The Fellowship of the Ring,Peter Jackson
The Lord of the Rings: The Two Towers,Peter Jackson


### 21. 從 `covid19` 資料庫的 `lookup_table` 資料表篩選出 `Country_Region` 名稱有 land 的國家，參考下列的預期查詢結果。

預期輸出：(10, 1) 的查詢結果。

In [ ]:
%%sql


Country_Region
Solomon Islands
New Zealand
Finland
Iceland
Ireland
Netherlands
Marshall Islands
Poland
Switzerland
Thailand
